In [ ]:
import requests
import pandas as pd
import json
ip = "http://127.0.0.1:8080" # can change here with ngrok url
requests_urls = {
    'upload_data': f"{ip}/ml/upload-book-vector",
    'update_data': f"{ip}/ml/update-book-vector",
}

data_frame = (pd.read_excel(r'/home/mitul/Projects/NBIC/DATA/nbic_raw_data.xlsx')).fillna('')
data_dict = data_frame.loc[0:4].to_dict('records')
print(len(data_dict))

In [ ]:
for data_lump in data_dict:
    print(requests_urls['upload_data'])
    payload = data_lump
    response = requests.post(url=requests_urls['upload_data'], json=payload)
    print(response.text)

# Upload background check

In [ ]:
import pinecone
import openai
import os
from langchain.document_loaders import JSONLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone

In [ ]:
# setting up os variables
os.environ['PINECONE_API_KEY'] = ""
os.environ['OPENAI_API_KEY'] = ""

In [ ]:
# initializing pinecone obj
pinecone.init(api_key=os.environ['PINECONE_API_KEY'],
              environment="us-west4-gcp-free")

In [ ]:
# Fetching relevant results as per query using pinecone CRUD functions

query = "The Powerful Purpose of Introverts: Why the World Needs You to Be You"

#generating embeddings of current query
pinecone_index = pinecone.Index('nbic')
response = openai.Embedding.create(api_key = os.environ['OPEN_API_KEY'],model="text-embedding-ada-002", input=query)
embedding_data = response["data"][0]["embedding"]


stats = pinecone_index.describe_index_stats()
namespace_map = stats['namespaces']
namespace_name = list(namespace_map.keys())[0]
result = pinecone_index.query(vector= embedding_data , top_k=10000, namespace=namespace_name,include_metadata=True, include_values=True)
result


In [ ]:
# Fetching relevant results as per query using Langchain's support functions

query = "The Powerful Purpose of Introverts: Why the World Needs You to Be You"
embeddings = OpenAIEmbeddings()
docsearch = Pinecone.from_existing_index('nbic',embeddings)
docs = docsearch.similarity_search(query,namespace='nbic',k=6)
docs

In [ ]:
# listing function to get all the embeddings
def list_all_embeddings(pinecone_index):
    stats = pinecone_index.describe_index_stats()
    namespace_map = stats['namespaces']
    ret = []
    for namespace in namespace_map:
        vector_count = namespace_map[namespace]['vector_count']
        res = pinecone_index.query(vector=[0 for _ in range(1536)], top_k=10000, namespace=namespace, include_metadata=True, include_values=True)
        for match in res['matches']:
            ret.append(match)
    return ret